A MACHINE LEARNING MODEL BUILT TO PREDICT STUDENT'S ACADEMIC PEROFRMANCE GIVEN THEIR INDIVIDUAL FEATURES

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import warnings 
warnings.filterwarnings('ignore')

: 